In [1]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import os
os.environ['PYTHONWARNINGS'] = "ignore"
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
import xgboost as xgb
from shap import GPUTreeExplainer
from matplotlib.ticker import MaxNLocator,MultipleLocator
from scipy.stats import kendalltau
from scipy.special import expit

import mlresearch
mlresearch.utils.set_matplotlib_style()
from mlresearch.utils import set_matplotlib_style
set_matplotlib_style(font_size=27)

In [2]:
import numpy as np
print(np.__version__) #1.26.4
# print(shap.__version__) #0.46.1.dev86
print(sklearn.__version__) #1.6.0
print(xgb.__version__) #1.7.6
import sys
print(sys.version)

2.0.2
1.6.1
2.1.4
3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:56:27) [GCC 11.2.0]


## Preprocessing

In [3]:
# Use White alone & African American only 
FEAT_CNT = 8
STATE = 'VA'
FOLDS = 5
seeds = [0,21,42,63,84]

In [4]:
categorical_cols =['Occupation', 'Marriage','Place of Birth','Sex', 'Race']

with open(file=f'dataset/ACS_Income_{STATE}.pickle', mode='rb') as f:
    df=pickle.load(f)
columns = df.columns
with pd.option_context('future.no_silent_downcasting', True):
    df.replace([' <=50K',' >50K'],
                 [0,1], inplace = True)
    df['Sex'].replace( {'Female':0.0},inplace = True)
    df['Sex'].replace({'Male':1.0}, inplace = True)
df['Race'] = df['Race'].str.strip().str.lower()
race_used = ['asian alone', 'black or african american alone', 'white alone']

# Replace values not in race_used with 'Other'
df['Race'] = df['Race'].apply(lambda x: x if x in race_used else 'other')
X = df.iloc[:, 0:FEAT_CNT]
Y = df.iloc[:, FEAT_CNT]

category_col =['Occupation', 'Marriage','Place of Birth', 'Race']
X = pd.get_dummies(X, columns=category_col, drop_first=False)
for c in X.columns:
    X[c] = X[c].astype(float)

In [5]:
'''
2 buckets
    White, Black + Asian + Other
    White + Black , Asian + Other ( Number-wise majority vs minority)
    White + Asian, Black + Other (Privileged vs unprivileged)
3 buckets
    White, Black, Asian + Other
    White, Asian, Black + Other
'''

white, black, asian, other =  'Race_white alone','Race_black or african american alone', 'Race_asian alone', 'Race_other'
all_buckets = {'W,B+A+O':[[white],[black,asian,other]], 'W+B,A+O':[[white,black],[asian,other]],"W+A,B+O":[[white,asian],[black,other]], 
           'W,B,A+O':[[white],[black],[asian,other]],'W,A,B+O': [[white],[asian],[black,other]] 
              }

## Utils

In [6]:
def assign_race(X, buckets):
    X2 = X.copy()
    for bucket in buckets:
        if len(bucket) > 1:
            # Determine majority race based on overall counts
            b_counts = {race: X2[race].sum() for race in bucket}
            max_race = max(b_counts, key=b_counts.get)
            
            # Identify rows that have any race indicator in the bucket
            mask = X2[bucket].sum(axis=1) > 0
            
            # Set all columns in the bucket to 0 for these rows
            X2.loc[mask, bucket] = 0.0
            
            # Then set the majority race column to 1 for these rows
            X2.loc[mask, max_race] = 1.0
    return X2

def sum_race_shaps(shap_vals):
    
    all_races = [white,black,asian,other]
    all_race_idx = [list(X.columns).index(race) for race in all_races]

    # Step 1: Compute the sum of specified columns for each row
    new_column =shap_vals[:, all_race_idx].sum(axis=1)
    
    # Step 2: Add the new column to the array
    shap_vals = np.hstack((shap_vals, new_column.reshape(-1, 1)))
    
    # Step 3: Delete the specified columns
    shap_vals = np.delete(shap_vals, all_race_idx, axis=1)
    return shap_vals
def get_ranks(shap_vals):

    sorted_indices = np.argsort(-np.abs(shap_vals), axis=1)  # Indices of absolute values in descending order
    rank = np.empty_like(sorted_indices)             # Create an empty array of the same shape
    rows, cols = shap_vals.shape                            # Get the shape of the array
    rank[np.arange(rows)[:, None], sorted_indices] = np.arange(1, cols + 1)  # Assign ranks row-wise

    target_rank = rank[:,-1]

    return rank,target_rank

def compute_shap(X_train,X_test,Y_train,Y_test,models, seed):

    print('**********START**********')
    # Extract the best model
    best_model = models[i]

    explainer = GPUTreeExplainer(best_model,X_train,feature_perturbation = 'interventional')
    shap_values = explainer(X_test)
    pred = best_model.predict(X_test)
    return shap_values


def compute_fidelity(pred, sv, base):

    sv_sums = expit(np.sum(sv, axis=1)+base)
    binary_predictions = (sv_sums > 0.5).astype(float)
    fidelity = np.mean(binary_predictions == pred)
    match_idx = np.where(binary_predictions == pred)[0]
        
    return fidelity,match_idx
            

## Train the model with plain Age

In [7]:
base_shap_vals = list()
base_preds = list()
base_accs = list() 
base_f1s = list()
base_ranks = list()
base_race_ranks = list()

base_firsts = list()
base_percentages = list()
base_first_race_shap_vals = list()

base_models = list()

for seed in tqdm(seeds):
    np.random.seed(seed)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    for train_val_idx, test_idx in splitter.split(X, Y):
        X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
        Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
    
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
        X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx]
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],  # Number of boosting rounds
        'classifier__max_depth': [3, 5, 7,9,11],          # Maximum tree depth
        'classifier__learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage 
        'classifier__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
        'classifier__gamma': [0, 0.1, 0.2],          # Minimum loss reduction for a split
    }
    model = xgb.XGBClassifier(random_state=seed)
    grid_search = GridSearchCV(
        model, 
        param_grid,              # 3-fold cross-validation
        scoring='f1',   # Evaluation metric
        n_jobs=13,            # Use all processors
        verbose=1             # Print progress
    )

    grid_search.fit(X_train, Y_train)
    
    # Extract the best model
    best_model = grid_search.best_estimator_
    base_models.append(best_model)
    explainer = GPUTreeExplainer(best_model,X_train, feature_perturbation='interventional') 
    shap_values = explainer(X_test)
    
    sv = sum_race_shaps(shap_values.values)
    
    base_rank,race_rank= get_ranks(sv)
    base_ranks.append(base_rank)
    
    pred = best_model.predict(X_test)
    # base_models.append(best_model)
    base_shap_vals.append(shap_values)
    base_preds.append(pred)
    base_accs.append(accuracy_score(Y_test,pred)*100)
    base_f1s.append(f1_score(Y_test,pred)*100)
    base_ranks.append(base_rank)
    base_race_ranks.append(race_rank)
    # First

    ## Indices of first
    first = [int(j) for j,v in enumerate(race_rank) if v == 1]
    base_firsts.append(first)
    base_percentages.append(len(first)/len(X_test) *100)
    base_first_race_shap_vals.append(race_rank[first])
print(f'Overall average acc: {sum(base_accs)/len(base_accs):.2f} average f1s : {sum(base_f1s)/len(base_f1s):.2f}')





  0%|                                                       | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 20%|█████████▍                                     | 1/5 [00:28<01:55, 28.98s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 40%|██████████████████▊                            | 2/5 [00:47<01:08, 22.72s/it]

...

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 80%|█████████████████████████████████████▌         | 4/5 [01:23<00:19, 19.54s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


100%|███████████████████████████████████████████████| 5/5 [01:41<00:00, 20.25s/it]

Overall average acc: 79.47 average f1s : 76.31


In [20]:
with open(path + '/Bucket_Attack_Income_base_race_ranks_cv.pickle', 'wb') as f:
    pickle.dump(base_race_ranks, f, pickle.HIGHEST_PROTOCOL)

 # Race Bucetkization

In [17]:
bucket_fids = list()
bucket_race_ranks = list()


for race in all_buckets.values():
    # compute bin edges as shap values for bucketized data
    b_fids = list()
    b_race_ranks = list()


    X2 = assign_race(X, race)
    for i, seed in enumerate(seeds):
        np.random.seed(seed)
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
        for train_val_idx, test_idx in splitter.split(X2, Y):
            X_train_val, X_test = X2.iloc[train_val_idx], X2.iloc[test_idx]
            Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
        
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
        for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx] 

        b_shap = compute_shap(X_train,X_test,Y_train,Y_test,base_models,seed)
        sv = sum_race_shaps(b_shap.values)
        b_rank, b_race_rank = get_ranks(sv)

        preds = base_preds[i]
        base = b_shap.base_values
        b_fid, b_agreed = compute_fidelity(preds,sv,base)

        b_fids.append(b_fid)
        b_race_ranks.append(b_race_rank)
        

    bucket_fids.append(b_fids)
    bucket_race_ranks.append(b_race_ranks)


  

**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********
**********START**********


In [19]:
import pickle
from pathlib import Path
path = './results'
if not os.path.exists(path):
   # Create a new directory because it does not exist
   os.makedirs(path)
with open(path + '/Bucket_Attack_Income_bucket_fids_cv.pickle', 'wb') as f:
    pickle.dump(bucket_fids, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_Attack_Income_bucket_race_ranks_cv.pickle', 'wb') as f:
    pickle.dump(bucket_race_ranks, f, pickle.HIGHEST_PROTOCOL)